# Исследование базы данных крупного сервиса для чтения книг по подписке

**Цель** - проанализировать базу данных. База данных содержит информацию о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

Задачи:

- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц для исключениях из анализа брошюры;
- Определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

**Описание данных:**

Таблица **books**
Содержит данные о книгах:

- *book_id* — идентификатор книги;
- *author_id* — идентификатор автора;
- *title* — название книги;
- *num_pages* — количество страниц;
- *publication_date* — дата публикации книги;
- *publisher_id* — идентификатор издателя.

Таблица **authors**
Содержит данные об авторах:

- *author_id* — идентификатор автора;
- *author* — имя автора.

Таблица **publishers**
Содержит данные об издательствах:

- *publisher_id* — идентификатор издательства;
- *publisher* — название издательства.

Таблица **ratings**
Содержит данные о пользовательских оценках книг:

- *rating_id* — идентификатор оценки;
- *book_id* — идентификатор книги;
- *username* — имя пользователя, оставившего оценку;
- *rating* — оценка книги.

Таблица **reviews**
Содержит данные о пользовательских обзорах на книги:

- *review_id* — идентификатор обзора;
- *book_id* — идентификатор книги;
- *username* — имя пользователя, написавшего обзор;
- *text* — текст обзора.

## Загрузка данных и ознакомление с таблицами

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Взглянем на данные поближе, выведем первые 5 строк.

In [3]:
def show(sql):
    return pd.io.sql.read_sql(sql, con = engine) 

In [4]:
query_books = '''
        SELECT * FROM books
        '''
books = show(query_books)

query_authors = '''
                SELECT * FROM authors
                '''
authors = show(query_authors)

query_publishers = '''
                   SELECT * FROM publishers
                   '''
publishers = show(query_publishers)

query_ratings = '''
                SELECT * FROM ratings
                '''
ratings = show(query_ratings)

query_reviews = '''
                SELECT * FROM reviews
                '''
reviews = show(query_reviews)

In [5]:
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [6]:
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [7]:
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [8]:
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [9]:
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Задачи:

### Количество книг, выпущенных после 1 января 2000 года

In [10]:
books_cnt = '''
            SELECT 
                COUNT(book_id) AS books_cnt 
            FROM 
                books
            WHERE 
                publication_date > '2000-01-01'
            '''
show(books_cnt)

,books_cnt
0,819


Количество книг, которые были опубликованы после 1 января 2000 года, составило 819 штук.

### Количество обзоров и средняя оценка каждой книги

In [11]:
reviews_cnt = '''
              SELECT 
                  books.title AS title,
                  books.book_id AS book_id,
                  books.publication_date AS pub_date, 
                  COUNT(DISTINCT reviews.review_id) AS reviews_cnt, 
                  AVG(ratings.rating) AS rating_avg
              FROM 
                  books
              LEFT JOIN ratings ON ratings.book_id = books.book_id
              LEFT JOIN reviews ON reviews.book_id = books.book_id
              GROUP BY
                  books.title, 
                  books.book_id
              ORDER BY
                  reviews_cnt DESC
              LIMIT 10
              '''
show(reviews_cnt)

,title,book_id,pub_date,reviews_cnt,rating_avg
0,Twilight (Twilight #1),948,2006-09-06,7,3.662500
1,The Da Vinci Code (Robert Langdon #2),696,2006-03-28,6,3.830508
2,Eat Pray Love,207,2007-02-01,6,3.395833
3,The Alchemist,627,1993-05-01,6,3.789474
4,The Catcher in the Rye,673,2001-01-30,6,3.825581
5,The Curious Incident of the Dog in the Night-Time,695,2004-05-18,6,4.081081
6,Harry Potter and the Prisoner of Azkaban (Harr...,302,2004-05-01,6,4.414634
7,Harry Potter and the Chamber of Secrets (Harry...,299,1999-06-02,6,4.287500
8,Outlander (Outlander #1),497,2005-07-26,6,4.125000
9,The Book Thief,656,2006-03-14,6,4.264151


Наибольшее количество обзоров у книги **Twilight** (Сумерки) 7 штук и её средняя оценка 3.66

### Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [12]:
publisher_popular = '''
                    SELECT  
                        publishers.publisher AS pub, 
                        COUNT(books.book_id) AS books_cnt
                    FROM
                        publishers
                        LEFT JOIN books ON books.publisher_id = publishers.publisher_id
                    WHERE
                        books.num_pages > 50  
                    GROUP BY
                        pub
                    ORDER BY
                        books_cnt DESC
                    LIMIT 1
                    '''
show(publisher_popular)                   

,pub,books_cnt
0,Penguin Books,42


Издательство Penguin Books выпустило 42 книги толще 50 листов.

### Автор с самой высокой средней оценкой книги (учтем только книги с 50 и более оценками)

In [15]:
author_rating = '''
                SELECT 
                    SUBQ.author AS author,  
                    AVG(SUBQ.rating_avg) AS rating 
                FROM (
                    SELECT
                        authors.author AS author,
                        books.book_id AS id_book, 
                        AVG(ratings.rating) AS rating_avg,
                        COUNT(ratings.rating) AS rating_cnt
                    FROM 
                        authors
                        INNER JOIN books ON books.author_id = authors.author_id
                        INNER JOIN ratings ON ratings.book_id = books.book_id
                    GROUP BY 
                        author,
                        id_book
                        HAVING
                       COUNT(ratings.rating) >= 50) AS SUBQ
                GROUP BY
                    author
                ORDER BY 
                    rating DESC
                LIMIT 10
                '''
show(author_rating)

,author,rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Автор J.K. Rowling/Mary GrandPré имеет самую высокую среднию оценку книг в 4.28, с учетом только книг с 50 и более оценками. 

### Среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [14]:
user_reviews = '''
               SELECT 
                   ROUND(AVG(SUBQ.reviews_cnt), 2)
               FROM(
                   SELECT
                       username,
                       COUNT(review_id) AS reviews_cnt
                   FROM
                       reviews
                   WHERE 
                       username IN (
                           SELECT
                               username
                           FROM
                               ratings
                           GROUP BY
                               username 
                           HAVING 
                           COUNT(ratings.rating_id) >= 50)
                   GROUP BY
                       username) AS SUBQ
                '''
show(user_reviews)

,round
0,24.22


## Вывод

- C 1 января 2000 года было выпущено после 819 книг.
- По количеству обзоров лидирует первая часть саги "Сумерки" с общим числом отзывов 7. 
- Наибольшее количество книг выпустило издательство Penguin Books - 42 книги.
- Автор J.K. Rowling/Mary GrandPré имеет самую высокую среднию оценку книг в 4.29.
- Среднее количество обзоров от пользователей, которые поставили более 50 оценок равно 24.